# 推荐系统算法对比实验

## BERTopic增强协同过滤 vs 传统协同过滤 vs 内容推荐

**数据集**: MovieLens-100k
**评估指标**: Precision@K, Recall@K, F1@K, MAE, NDCG@K, Coverage@K

优化点：
- 添加随机基线和流行度基线
- 修复exclude逻辑，确保所有模型统一排除已交互物品
- MAE计算改为全局test集预测
- 添加冷启动子集评估
- 添加NDCG和Coverage指标
- 数据划分使用时间序，确保真实性
---

## 1. 环境配置

In [5]:
# @title 安装依赖
!pip install pandas numpy matplotlib seaborn scikit-learn sentence-transformers bertopic -q

## 2. 导入库

In [6]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import ndcg_score
import warnings
import time
import urllib.request
import zipfile
# ================== 在这个 cell 最下面添加 ==================
import re
import jieba

# 修复 BERTopic 相关的导入错误
from sentence_transformers import SentenceTransformer as st_bert
from bertopic import BERTopic as bertopic_bert
from sklearn.feature_extraction.text import CountVectorizer as count_vectorizer
import json
import numpy as np_bert # BERTopic类中使用了np_bert，故此处显式导入
from datetime import datetime # 用于时间衰减功能
from sklearn.decomposition import NMF # 用于NMF分解
# =========================================================

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 3. 数据下载与加载

In [7]:
def download_movielens_100k_to_base_dir(base_extract_dir):
    """自动下载MovieLens-100k数据集，并解压到指定基础目录"""
    os.makedirs(base_extract_dir, exist_ok=True)

    # Expected final path after extraction
    final_data_folder = os.path.join(base_extract_dir, "ml-100k")

    if os.path.exists(os.path.join(final_data_folder, "u.data")):
        print(f"数据集已存在: {final_data_folder}")
        return True

    print("正在下载 MovieLens-100k...")
    url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
    zip_path = os.path.join(base_extract_dir, "ml-100k.zip")

    try:
        urllib.request.urlretrieve(url, zip_path)
        print("下载完成，正在解压...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(base_extract_dir) # Extracts ml-100k/u.data to base_extract_dir/ml-100k/u.data
        os.remove(zip_path)
        print("数据集准备完成!")
        return True
    except Exception as e:
        print(f"下载失败: {e}")
        return False

def load_movielens_100k(data_folder_path):
    """加载MovieLens-100k数据集"""
    print("=" * 60)
    print("1. 加载数据集")
    print("=" * 60)

    # Check for the existence of u.data in the provided data_folder_path
    if not os.path.exists(os.path.join(data_folder_path, "u.data")):
        print(f"数据集不存在于 {data_folder_path}，尝试下载...")
        # Determine the base directory for download (parent of data_folder_path)
        base_dir_for_download = os.path.dirname(data_folder_path)
        if not base_dir_for_download: # Handle case where data_folder_path is just "ml-100k"
            base_dir_for_download = "."

        download_movielens_100k_to_base_dir(base_dir_for_download)

        # After attempting download, check again
        if not os.path.exists(os.path.join(data_folder_path, "u.data")):
            raise FileNotFoundError(
                f"Failed to find u.data at {data_folder_path}/u.data after download attempt."
            )

    ratings_cols = ["user_id", "item_id", "rating", "timestamp"]
    ratings_df = pd.read_csv(
        os.path.join(data_folder_path, "u.data"), sep="\t", names=ratings_cols, encoding="latin-1"
    )

    items_cols = ["item_id", "title", "release_date", "video_release_date", "imdb_url"] + \
                 [f"genre_{i}" for i in range(19)]
    items_df = pd.read_csv(
        os.path.join(data_folder_path, "u.item"), sep="|", names=items_cols, encoding="latin-1"
    )

    print(f"评分数据: {ratings_df.shape}")
    print(f"电影数据: {items_df.shape}")

    return ratings_df, items_df

# 数据目录
# This DATA_DIR_FINAL should be the actual directory containing u.data, which is ml-100k inside the base download dir
DATA_DIR_FINAL = "./data/ml-100k" # Expected path to ml-100k folder after extraction

ratings_df, items_df = load_movielens_100k(DATA_DIR_FINAL)

1. 加载数据集
评分数据: (100000, 4)
电影数据: (1682, 24)


## 4. 数据预处理

In [8]:
GENRE_NAMES = [
    "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical",
    "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western", "Other"
]

def preprocess_data(ratings_df, items_df):
    """预处理数据"""
    print("\n2. 数据预处理")
    print("-" * 60)

    genre_cols = [f"genre_{i}" for i in range(19)]

    def get_genres(row):
        genres = []
        for i, col in enumerate(genre_cols):
            if row[col] == 1 and i < len(GENRE_NAMES):
                genres.append(GENRE_NAMES[i])
        return " ".join(genres)

    items_df["genres"] = items_df.apply(get_genres, axis=1)
    items_df["content"] = items_df["title"] + " " + items_df["genres"]

    poems = items_df.apply(
        lambda row: {"id": row["item_id"], "content": row["content"], "title": row["title"]},
        axis=1
    ).tolist()

    interactions = ratings_df.apply(
        lambda row: {
            "user_id": row["user_id"],
            "poem_id": row["item_id"],
            "rating": float(row["rating"]),
            "created_at": pd.to_datetime(row["timestamp"], unit="s"),
        },
        axis=1
    ).tolist()

    print(f"电影数: {len(poems)}")
    print(f"交互数: {len(interactions)}")

    return poems, interactions

poems, interactions = preprocess_data(ratings_df, items_df)


2. 数据预处理
------------------------------------------------------------
电影数: 1682
交互数: 100000


## 5. 数据集划分（时间序优化）

In [9]:
def split_train_test_time_based(interactions, test_ratio=0.2):
    """时间序划分训练集和测试集"""
    train_interactions = []
    test_interactions = []

    user_groups = defaultdict(list)
    for inter in interactions:
        user_groups[inter["user_id"]].append(inter)

    for user_id, user_interactions in user_groups.items():
        user_interactions = sorted(user_interactions, key=lambda x: x["created_at"])
        n = len(user_interactions)
        train_size = int(n * (1 - test_ratio))
        train_interactions.extend(user_interactions[:train_size])
        test_interactions.extend(user_interactions[train_size:])

    print(f"\n3. 数据集划分")
    print("-" * 60)
    print(f"训练集: {len(train_interactions)} 条")
    print(f"测试集: {len(test_interactions)} 条")

    return train_interactions, test_interactions

train_interactions, test_interactions = split_train_test_time_based(interactions)


3. 数据集划分
------------------------------------------------------------
训练集: 79619 条
测试集: 20381 条


## 6. 数据过滤 (加速，同时保留冷启动子集)

In [10]:
MIN_RATINGS = 10  # 降低阈值以保留更多冷启动

print("\n4. 数据过滤 (加速)")
print("-" * 60)

user_counts = pd.Series([i["user_id"] for i in train_interactions]).value_counts()
active_users = set(user_counts[user_counts >= MIN_RATINGS].index.tolist())
cold_users = set(user_counts[user_counts < MIN_RATINGS].index.tolist())  # 保留冷启动用户

item_counts = pd.Series([i["poem_id"] for i in train_interactions]).value_counts()
popular_items = set(item_counts[item_counts >= MIN_RATINGS].index.tolist())
long_tail_items = set(item_counts[item_counts < MIN_RATINGS].index.tolist())  # 保留长尾

train_filtered = [
    i for i in train_interactions
    if i["user_id"] in active_users and i["poem_id"] in popular_items
]
poems_filtered = [p for p in poems if p["id"] in popular_items]

print(f"过滤后训练集: {len(train_filtered)} 条")
print(f"活跃用户: {len(active_users)}")
print(f"热门电影: {len(popular_items)}")
print(f"冷启动用户: {len(cold_users)}")
print(f"长尾电影: {len(long_tail_items)}")


4. 数据过滤 (加速)
------------------------------------------------------------
过滤后训练集: 77289 条
活跃用户: 943
热门电影: 1019
冷启动用户: 0
长尾电影: 594


## 7. 算法实现（核心区域留空）

In [11]:
# 核心算法实现留空，请在此处填充你的 ItemBasedCFRecommender, ContentBasedRecommender, BERTopicEnhancedCF 类定义
# 示例模板：
class ItemBasedCFRecommender:
    """
    基于物品的协同过滤算法 (Item-Based Collaborative Filtering)
    使用评分矩阵计算物品相似度进行推荐
    """

    def __init__(self, k_neighbors=30):
        self.k_neighbors = k_neighbors
        self.item_similarity = None
        self.rating_matrix = None
        self.poem_id_to_idx = None
        self.idx_to_poem_id = None

    def fit(self, interactions, poem_ids):
        """
        训练模型：构建评分矩阵和物品相似度矩阵

        Args:
            interactions: list of dict, each contains 'user_id', 'poem_id', 'rating', 'created_at'
            poem_ids: list of all poem ids
        """
        self.poem_id_to_idx = {pid: idx for idx, pid in enumerate(poem_ids)}
        self.idx_to_poem_id = {idx: pid for pid, idx in self.poem_id_to_idx.items()}

        users = set(i["user_id"] for i in interactions)
        user_id_to_idx = {uid: idx for idx, uid in enumerate(users)}

        n_users = len(users)
        n_items = len(poem_ids)

        self.rating_matrix = np.zeros((n_users, n_items))

        for inter in interactions:
            u_idx = user_id_to_idx[inter["user_id"]]
            p_idx = self.poem_id_to_idx[inter["poem_id"]]
            self.rating_matrix[u_idx, p_idx] = inter.get("rating", 3.0)

        self._compute_similarity()

        print(f"[Item-CF] 评分矩阵构建完成: {self.rating_matrix.shape}")
        print(
            f"[Item-CF] 矩阵密度: {(self.rating_matrix > 0).sum() / (n_users * n_items):.2%}"
        )

    def _compute_similarity(self):
        """计算物品相似度矩阵（向量化，替代O(n²)Python循环）"""
        # 均值中心化：对每列（物品）只用有评分的用户
        R = self.rating_matrix.copy()
        # 按列减去各物品均值（只考虑评过分的用户）
        col_mask = (R > 0)
        col_sums = R.sum(axis=0)
        col_counts = col_mask.sum(axis=0)
        col_means = np.where(col_counts > 0, col_sums / np.maximum(col_counts, 1), 0)
        R_centered = np.where(col_mask, R - col_means, 0)
        # 用 sklearn cosine_similarity 一次性计算
        self.item_similarity = cosine_similarity(R_centered.T)
        # 保证对角线为1，负值截断为0（可选策略）
        np.fill_diagonal(self.item_similarity, 1.0)
        print(f"[Item-CF] 相似度矩阵构建完成（向量化）")

    def get_user_ratings(self, user_id, user_interactions):
        """获取用户评分向量"""
        n_items = len(self.poem_id_to_idx)
        ratings = np.zeros(n_items)

        for inter in user_interactions:
            if inter["poem_id"] in self.poem_id_to_idx:
                p_idx = self.poem_id_to_idx[inter["poem_id"]]
                ratings[p_idx] = inter.get("rating", 3.0)

        return ratings

    def recommend(self, user_interactions, exclude_ids=None, top_k=10):
        """
        为用户推荐诗歌

        Args:
            user_interactions: list of dicts with 'poem_id', 'rating'
            exclude_ids: set of poem ids to exclude
            top_k: number of recommendations

        Returns:
            list of dict: recommended poems with scores
        """
        exclude_ids = exclude_ids or set()

        user_ratings = self.get_user_ratings(None, user_interactions)
        rated_items = np.where(user_ratings > 0)[0]

        if len(rated_items) == 0:
            return []

        scores = np.zeros(len(self.poem_id_to_idx))

        for item_idx in range(len(self.poem_id_to_idx)):
            if user_ratings[item_idx] > 0:
                continue

            neighbors = self.item_similarity[item_idx, rated_items]
            neighbor_ratings = user_ratings[rated_items]

            pos_mask = neighbors > 0
            if pos_mask.sum() > 0:
                scores[item_idx] = np.dot(
                    neighbors[pos_mask], neighbor_ratings[pos_mask]
                ) / (np.abs(neighbors[pos_mask]).sum() + 1e-8)

        results = []
        for idx, score in enumerate(scores):
            poem_id = self.idx_to_poem_id[idx]
            if poem_id not in exclude_ids and user_ratings[idx] == 0:
                results.append({"poem_id": poem_id, "score": float(score), "title": ""})

        results.sort(key=lambda x: x["score"], reverse=True)
        return results[:top_k]

    def predict_rating(self, user_interactions, poem_id):
        """预测用户对物品的评分"""
        if poem_id not in self.poem_id_to_idx:
            return 3.0

        poem_idx = self.poem_id_to_idx[poem_id]
        user_ratings = self.get_user_ratings(None, user_interactions)
        rated_items = np.where(user_ratings > 0)[0]

        if len(rated_items) == 0:
            return 3.0

        neighbors = self.item_similarity[poem_idx, rated_items]
        neighbor_ratings = user_ratings[rated_items]

        pos_mask = neighbors > 0
        if pos_mask.sum() > 0:
            return np.clip(
                np.dot(neighbors[pos_mask], neighbor_ratings[pos_mask])
                / (np.abs(neighbors[pos_mask]).sum() + 1e-8),
                1.0,
                5.0,
            )
        return 3.0

    def predict_all_ratings(self, user_interactions):
        """预测用户对所有物品的评分"""
        n_items = len(self.poem_id_to_idx)
        user_ratings = self.get_user_ratings(None, user_interactions)
        rated_items = np.where(user_ratings > 0)[0]

        if len(rated_items) == 0:
            return np.full(n_items, 3.0)

        predictions = np.zeros(n_items)

        for item_idx in range(n_items):
            if user_ratings[item_idx] > 0:
                predictions[item_idx] = user_ratings[item_idx]
                continue

            neighbors = self.item_similarity[item_idx, rated_items]
            neighbor_ratings = user_ratings[rated_items]

            pos_mask = neighbors > 0
            if pos_mask.sum() > 0:
                predictions[item_idx] = np.clip(
                    np.dot(neighbors[pos_mask], neighbor_ratings[pos_mask])
                    / (np.abs(neighbors[pos_mask]).sum() + 1e-8),
                    1.0,
                    5.0,
                )
            else:
                predictions[item_idx] = 3.0

        return predictions

class ContentBasedRecommender:
    """
    基于内容的推荐 (Content-Based Filtering)

    工作流程：
    1. TF-IDF向量化：把所有物品的内容（标题+简介+类型）转为TF-IDF向量
    2. 用户画像：根据用户历史交互过的物品，构建用户偏好向量
       - 用户画像 = 加权平均(物品TF-IDF向量 × 用户评分权重)
    3. 推荐：计算用户画像与所有物品的相似度，返回top-k
    """

    def __init__(self, max_features=1000, ngram_range=(1, 2)):
        """
        Args:
            max_features: TF-IDF最大特征数
            ngram_range: n-gram范围
        """
        self.max_features = max_features
        self.ngram_range = ngram_range

        self.tfidf_vectorizer = None
        self.tfidf_matrix = None
        self.items = None
        self.global_mean = 3.0
        self.item_id_to_idx = None

    def _tokenize(self, text):
        """
        中英文分词

        Args:
            text: 待分词文本

        Returns:
            分词后的文本
        """
        if not text:
            return ""

        # 中文分词
        chinese_only = re.sub(r"[^\u4e00-\u9fa5\s]", "", text)
        chinese_words = jieba.lcut(chinese_only)
        chinese_words = [w for w in chinese_words if len(w) > 1]

        # 英文分词
        english_words = re.findall(r"[a-zA-Z]+", text.lower())

        return " ".join(chinese_words + english_words)

    def fit(self, items, interactions=None):
        """
        训练模型：构建TF-IDF特征矩阵

        Args:
            items: 物品列表，每个物品是dict，需包含 'id' 和 'content' 字段
            interactions: 交互列表，用于计算全局平均评分（可选）
        """
        self.items = items

        # 计算全局平均评分
        if interactions:
            ratings = [i.get("rating", 3.0) for i in interactions]
            self.global_mean = np.mean(ratings)

        # 构建内容文本
        contents = [self._tokenize(item.get("content", "")) for item in items]

        # TF-IDF向量化
        self.tfidf_vectorizer = TfidfVectorizer(
            max_features=self.max_features,
            ngram_range=self.ngram_range,
            min_df=1,
            stop_words="english",
        )
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(contents)
        self.item_id_to_idx = {item["id"]: idx for idx, item in enumerate(self.items)}
        # ★ Fix: 预缓存稠密矩阵，避免 recommend/predict_rating 每次 toarray()
        self.tfidf_dense = self.tfidf_matrix.toarray()
        print(f"[ContentBased] 物品ID到索引映射构建完成: {len(self.item_id_to_idx)} 条")
        print(f"[ContentBased] TF-IDF矩阵: {self.tfidf_matrix.shape}")


    def _build_user_profile(self, user_interactions):
        """
        构建用户画像向量

        公式: user_profile = Σ(物品向量 × 评分权重) / Σ(评分权重)
        评分权重 = |rating - 3|  (评分越高权重越大)

        Args:
            user_interactions: 用户交互列表，每个包含 'poem_id' 和 'rating'

        Returns:
            用户画像向量 (numpy array)
        """
        if not user_interactions:
            return None

        # 收集用户评过分的物品
        rated_items = []
        weights = []

        for interaction in user_interactions:
            item_id = interaction.get("poem_id")
            rating = interaction.get("rating", 3.0)

            idx = self.item_id_to_idx.get(item_id)
            if idx is not None:
              rated_items.append(idx)
              weights.append(abs(rating - 3.0) + 0.5)

        if not rated_items:
            return None

        # ★ 使用预缓存 dense 矩阵，避免重复 toarray()
        rated_vectors = self.tfidf_dense[rated_items]    # already numpy array
        weights = np.array(weights)
        user_profile = np.average(rated_vectors, axis=0, weights=weights)
        return user_profile

    def recommend(
        self, user_interactions, all_interactions=None, exclude_ids=None, top_k=10
    ):
        """
        为用户推荐物品

        Args:
            user_interactions: 用户历史交互列表
            all_interactions: 所有交互（未使用，保留参数兼容）
            exclude_ids: 排除的物品ID集合
            top_k: 推荐数量

        Returns:
            推荐列表，每个元素是 dict: {"poem_id": id, "score": score}
        """
        exclude_ids = set(exclude_ids or [])

        # 构建用户画像
        user_profile = self._build_user_profile(user_interactions)

        if user_profile is None:
            return []

        # 计算用户画像与所有物品的相似度
        # ★ Fix: 使用预缓存的稠密矩阵
        similarities = cosine_similarity(
            user_profile.reshape(1, -1), self.tfidf_dense
        )[0]

        # 生成推荐列表
        results = []
        for idx, sim in enumerate(similarities):
            item = self.items[idx]
            item_id = item["id"]

            # 排除已交互的物品
            if item_id in exclude_ids:
                continue

            # 将相似度转换为预测评分
            # 公式: predicted_rating = global_mean + similarity * scale
            predicted_rating = self.global_mean + sim * 1.0
            predicted_rating = np.clip(predicted_rating, 1.0, 5.0)

            results.append({"poem_id": item_id, "score": float(predicted_rating)})

        # 按评分降序排列
        results.sort(key=lambda x: x["score"], reverse=True)

        return results[:top_k]

    def predict_rating(self, user_interactions, item_id):
        """单物品预测（用 predict_all_ratings 批量版更快）"""
        item_idx = self.item_id_to_idx.get(item_id)
        if item_idx is None:
            return self.global_mean
        profile = self._build_user_profile(user_interactions)
        if profile is None:
            return self.global_mean
        # ★ 直接用 tfidf_dense 行做点积（已 L2 normalize 的 TF-IDF 余弦）
        sim = float(self.tfidf_dense[item_idx] @ profile /
                    (np.linalg.norm(self.tfidf_dense[item_idx]) *
                     np.linalg.norm(profile) + 1e-8))
        return float(np.clip(self.global_mean + sim, 1.0, 5.0))

    def predict_all_ratings(self, user_interactions):
        """
        ★ 批量预测全部物品评分（MAE 评估时用此方法，比逐个调 predict_rating 快 100x）
        构建一次 profile → 一次矩阵乘法搞定所有物品
        """
        profile = self._build_user_profile(user_interactions)
        if profile is None:
            return np.full(len(self.items), self.global_mean)
        p_norm = np.linalg.norm(profile) + 1e-8
        # tfidf_dense 行已经是单位向量（TF-IDF 默认 L2 norm），直接点积
        row_norms = np.linalg.norm(self.tfidf_dense, axis=1) + 1e-8
        sims = (self.tfidf_dense @ profile) / (row_norms * p_norm)
        preds = np.clip(self.global_mean + sims, 1.0, 5.0)
        return preds

    def get_similar_items(self, item_id, top_k=10):
        """
        获取与指定物品最相似的物品（用于物品详情页）

        Args:
            item_id: 物品ID
            top_k: 返回数量

        Returns:
            相似物品列表
        """
        # 找到物品索引
        item_idx = None
        for idx, item in enumerate(self.items):
            if item["id"] == item_id:
                item_idx = idx
                break

        if item_idx is None:
            return []

        # 计算与所有物品的相似度
        similarities = cosine_similarity(
            self.tfidf_matrix[item_idx], self.tfidf_matrix
        )[0]

        # 排序并返回top-k（排除自身）
        similar_indices = np.argsort(similarities)[::-1]

        results = []
        for idx in similar_indices:
            if idx == item_idx:
                continue
            item = self.items[idx]
            results.append({"poem_id": item["id"], "score": float(similarities[idx])})
            if len(results) >= top_k:
                break

        return results

class BERTopicEnhancedCF:
    """
    语义感知协同过滤 (Semantic-Aware CF)

    ── 解决传统CF的深层动机盲区 ──────────────────────────────────────
    传统问题：用户A和用户B都给《静夜思》打5分。
      · 用户A：喜欢"思乡"情感，后续偏好大量思乡诗
      · 用户B：喜欢"月亮"意象，后续偏好大量写月诗
    传统CF：因共同评分高 → 认为两人相似 → 推荐完全一样的内容 ✗

    ── 本算法的解决方案 ─────────────────────────────────────────────
    1. [语义编码] 用 sentence-transformers 把每个物品映射到高维语义空间
       每个维度对应一种潜在语义概念（思乡、月亮、爱情、战争…）

    2. [用户主题偏好画像] 对每个用户，构建"主题偏好向量"：
       profile = Σ (rating - global_mean) × item_embedding
       · 高分物品的语义方向被强化（这就是你喜欢的）
       · 低分物品的语义方向被压制（这是你不喜欢的）
       用户A → profile指向"思乡"维度
       用户B → profile指向"月亮"维度
       两人的profile在语义空间中方向不同！

    3. [语义推荐] score(item) = cosine(user_profile, item_embedding)
       直接衡量物品语义与用户深层偏好的匹配度

    4. [冷启动] 只需少量评分即可建立profile，无需大量共同评分邻居

    5. [自适应融合] 冷用户→更多语义；热用户→更多CF协同信号
    """

    def __init__(self, cf_weight=0.35, semantic_weight=0.65, n_neighbors=30):
        self.cf_weight = cf_weight
        self.semantic_weight = semantic_weight
        self.n_neighbors = n_neighbors

        self.poems = []
        self.interactions = []
        self.poem_ids = []
        self.poem_id_map = {}

        self.rating_matrix = None       # (users, items)
        self.user_id_map = {}
        self.idx_to_user_id = {}
        self.global_mean = 3.0

        self.item_embeddings = None     # (items, dim) — sentence embeddings
        self.topic_matrix = None        # alias for item_embeddings

        # Precomputed at fit() time
        self.item_semantic_sim = None   # (items, items)
        self.cf_item_sim = None         # (items, items) adjusted-cosine
        self.enhanced_sim = None        # (items, items) fused
        self.user_topic_profiles = None # (users, dim) — 用户主题偏好画像

        self.embedding_model = None
        self.cache_dir = os.path.join(os.getcwd(), "data", "cache")
        os.makedirs(self.cache_dir, exist_ok=True)

    # ═══════════════════════════════════════════════════════ fit pipeline ══

    def fit(self, poems, interactions):
        self.poems = poems
        self.interactions = interactions
        self.poem_ids = [p["id"] for p in poems]
        self.poem_id_map = {pid: idx for idx, pid in enumerate(self.poem_ids)}

        print("[SemanticCF] Step 1/5  构建评分矩阵...")
        self._build_rating_matrix(interactions)

        print("[SemanticCF] Step 2/5  生成物品语义向量...")
        self._build_embeddings(poems)

        print("[SemanticCF] Step 3/5  计算Item-CF相似度（向量化）...")
        self._build_cf_similarity()

        if self.item_embeddings is not None:
            print("[SemanticCF] Step 4/5  构建用户主题偏好画像...")
            self._build_user_topic_profiles()
            print("[SemanticCF] Step 5/5  融合相似度矩阵...")
            self.item_semantic_sim = cosine_similarity(self.item_embeddings)
            self._fuse_similarities()
        else:
            print("[SemanticCF] Step 4-5 跳过（无语义向量），使用纯CF")
            self.enhanced_sim = self.cf_item_sim

        print("[SemanticCF] 训练完成 ✓")

    # ─────────────────────────────────────────── Step 1 ──────────────────

    def _build_rating_matrix(self, interactions):
        users = sorted(set(i["user_id"] for i in interactions))
        self.user_id_map = {uid: idx for idx, uid in enumerate(users)}
        self.idx_to_user_id = {idx: uid for uid, idx in self.user_id_map.items()}

        n_u, n_i = len(users), len(self.poem_ids)
        self.rating_matrix = np.zeros((n_u, n_i))
        all_ratings = []
        for inter in interactions:
            u = self.user_id_map.get(inter["user_id"])
            p = self.poem_id_map.get(inter["poem_id"])
            if u is not None and p is not None:
                r = inter.get("rating", 3.0)
                self.rating_matrix[u, p] = r
                all_ratings.append(r)
        self.global_mean = float(np.mean(all_ratings)) if all_ratings else 3.0
        print(f"  评分矩阵: {self.rating_matrix.shape}, "
              f"密度: {(self.rating_matrix > 0).mean():.2%}, "
              f"全局均值: {self.global_mean:.2f}")

    # ─────────────────────────────────────────── Step 2 ──────────────────

    def _build_embeddings(self, poems):
        """加载或计算物品语义向量（带磁盘缓存）"""
        emb_path = os.path.join(self.cache_dir, "sem_embeddings.npy")
        ids_path  = os.path.join(self.cache_dir, "sem_ids.json")

        if os.path.exists(emb_path) and os.path.exists(ids_path):
            try:
                with open(ids_path) as f:
                    cached_ids = json.load(f)
                if cached_ids == self.poem_ids:
                    self.item_embeddings = np.load(emb_path)
                    self.topic_matrix    = self.item_embeddings
                    print(f"  ✓ 从缓存加载 {len(self.poem_ids)} 个向量")
                    return
            except Exception:
                pass

        try:
            device = "cpu"
            try:
                import torch
                if torch.cuda.is_available():
                    device = "cuda"
                    print(f"  GPU: {torch.cuda.get_device_name(0)}")
            except Exception:
                pass

            print(f"  加载 sentence-transformer 模型 (device={device})...")
            self.embedding_model = st_bert(
                "paraphrase-multilingual-MiniLM-L12-v2", device=device
            )
            contents = [p.get("content", "") for p in poems]
            print(f"  编码 {len(contents)} 个物品...")
            self.item_embeddings = self.embedding_model.encode(
                contents, batch_size=128, show_progress_bar=True,
                convert_to_numpy=True, normalize_embeddings=True
            )
            self.topic_matrix = self.item_embeddings

            np.save(emb_path, self.item_embeddings)
            with open(ids_path, "w") as f:
                json.dump(self.poem_ids, f)
            print("  向量已缓存到磁盘")
        except Exception as e:
            print(f"  ⚠ 向量生成失败: {e}")
            self.item_embeddings = None

    # ─────────────────────────────────────────── Step 3 ──────────────────

    def _build_cf_similarity(self):
        """Adjusted-cosine item similarity（向量化，一次计算完毕）"""
        R    = self.rating_matrix                          # (users, items)
        mask = R > 0
        col_counts = mask.sum(axis=0).clip(min=1)
        col_means  = R.sum(axis=0) / col_counts
        R_c = np.where(mask, R - col_means[np.newaxis, :], 0.0)
        self.cf_item_sim = cosine_similarity(R_c.T)        # (items, items)
        np.fill_diagonal(self.cf_item_sim, 1.0)
        print(f"  Item-CF 相似度矩阵: {self.cf_item_sim.shape}")

    # ─────────────────────────────────────────── Step 4 ──────────────────

    def _build_user_topic_profiles(self):
        """
        核心：为每个用户构建语义主题偏好向量。

        profile_u = normalize( Σ_i  w_i · emb(i) )

        其中 w_i = rating_i - global_mean
          · w_i > 0  → 用户喜欢此物品的语义方向（加强）
          · w_i < 0  → 用户不喜欢此方向（抵消）
          · w_i ≈ 0  → 中立，微弱正向信号

        这样，用户A（喜欢思乡诗）的 profile 会在"思乡"维度上有大正值，
        用户B（喜欢写月诗）的 profile 会在"月亮"维度上有大正值，
        即使他们都给《静夜思》高分，两人的 profile 方向也不同。
        """
        n_u, dim = self.rating_matrix.shape[0], self.item_embeddings.shape[1]
        profiles = np.zeros((n_u, dim))

        for u in range(n_u):
            rated_idx = np.where(self.rating_matrix[u] > 0)[0]
            if len(rated_idx) == 0:
                continue
            ratings = self.rating_matrix[u, rated_idx]
            weights = ratings - self.global_mean
            # 中立评分给一点正向信号
            weights = np.where(np.abs(weights) < 0.3, 0.15, weights)

            embs = self.item_embeddings[rated_idx]         # (k, dim)
            # item_embeddings 已经 L2-normalized，直接加权即可
            profile = weights @ embs                       # (dim,)

            norm = np.linalg.norm(profile)
            profiles[u] = profile / norm if norm > 1e-8 else profile

        self.user_topic_profiles = profiles
        print(f"  用户主题画像: {profiles.shape}, "
              f"非零用户: {(np.linalg.norm(profiles, axis=1) > 1e-8).sum()}")

    # ─────────────────────────────────────────── Step 5 ──────────────────

    def _fuse_similarities(self):
        """Min-max 归一化后加权融合 CF + 语义相似度"""
        def minmax(m):
            lo, hi = m.min(), m.max()
            return (m - lo) / (hi - lo + 1e-8)

        cf_norm  = minmax(self.cf_item_sim)
        sem_norm = minmax(self.item_semantic_sim)
        self.enhanced_sim = self.cf_weight * cf_norm + self.semantic_weight * sem_norm
        np.fill_diagonal(self.enhanced_sim, 1.0)
        print(f"  融合矩阵: {self.cf_weight:.2f}×CF + {self.semantic_weight:.2f}×语义")

    # ═══════════════════════════════ Semantic User-CF ═══════════════════

    def _user_cf_semantic_scores(self, u_profile, user_ratings, exclude_set):
        """
        语义邻域 User-CF —— 这是超越纯 Content-Based 的关键。

        CB 只问：「这个物品的内容像不像你喜欢的内容？」
        语义 User-CF 问：「和你口味相似的用户，实际上给哪些物品打了高分？」

        两个信号完全独立，融合后能发现：
          · 用户 A 喜欢 [思乡] 主题 → profile 向量相似
          · 用户 A 还爱 [边塞诗] → 但你之前根本没读过，CB 不会推
          · 语义 User-CF 会：因为 A 的 profile 和你接近，A 的高分物品就成为你的推荐

        步骤：
          1. 全训练用户的 profile 矩阵 @ 当前用户 profile → 相似度向量
          2. Top-K 邻居（过滤相似度 < 阈值）
          3. 邻居评分矩阵加权聚合 → 每个物品的协同分数
        """
        if u_profile is None or self.user_topic_profiles is None:
            return None

        profile_norm = np.linalg.norm(u_profile)
        if profile_norm < 1e-8:
            return None

        # profile 已归一化，直接点积 = 余弦相似度  (n_users,)
        sims = self.user_topic_profiles @ u_profile          # (n_users,)
        sims = np.clip(sims, 0, None)                        # 只用正相关邻居

        # Top-K：取相似度最高的 n_neighbors 个
        top_idx = np.argsort(sims)[::-1][:self.n_neighbors]
        top_sims = sims[top_idx]
        valid = top_sims > 0.1                               # 相似度门槛
        top_idx, top_sims = top_idx[valid], top_sims[valid]

        if len(top_idx) == 0:
            return None

        # 加权聚合邻居评分：(K,) × (K, items) → (items,)
        neigh_R   = self.rating_matrix[top_idx]              # (K, items)
        w         = top_sims[:, np.newaxis]                  # (K, 1)
        has_rated = (neigh_R > 0).astype(np.float32)        # (K, items)

        weighted_sum = (w * neigh_R).sum(axis=0)            # (items,)
        sim_total    = (w * has_rated).sum(axis=0)          # (items,)

        scores = np.where(sim_total > 0, weighted_sum / sim_total, -np.inf)

        # mask exclude + already rated
        for pid in exclude_set:
            p = self.poem_id_map.get(pid)
            if p is not None:
                scores[p] = -np.inf
        scores[user_ratings > 0] = -np.inf
        return scores

    def predict_all_ratings(self, user_interactions):
        """批量预测（供 evaluate_recommender 的 MAE 计算使用）"""
        _, user_ratings, u_profile = self._get_user_state(user_interactions)
        n = len(self.poem_ids)
        preds = np.full(n, self.global_mean)

        # CF 路径
        rated = np.where(user_ratings > 0)[0]
        if len(rated) > 0:
            sims = self.enhanced_sim[:, rated]
            r_v  = user_ratings[rated]
            pos  = sims > 0
            w_sum   = np.where(pos, sims * r_v, 0.0).sum(axis=1)
            sim_sum = np.where(pos, np.abs(sims), 0.0).sum(axis=1) + 1e-8
            cf_preds = np.clip(w_sum / sim_sum, 1.0, 5.0)
            preds = np.where(sim_sum > 1e-4, cf_preds, preds)

        # 已有评分照原值
        preds[rated] = user_ratings[rated]
        return preds

    # ══════════════════════════════════════════════════════ recommend ══════

    def _get_user_state(self, user_interactions):
        """从交互历史提取 (u_idx, user_ratings, u_topic_profile)"""
        u_idx = self.user_id_map.get(
            user_interactions[0]["user_id"] if user_interactions else None
        )
        user_ratings = np.zeros(len(self.poem_ids))
        for inter in user_interactions:
            p = self.poem_id_map.get(inter["poem_id"])
            if p is not None:
                user_ratings[p] = inter.get("rating", 3.0)

        # 主题画像：训练用户取预计算结果，否则实时构建
        if u_idx is not None and self.user_topic_profiles is not None:
            u_profile = self.user_topic_profiles[u_idx]
        elif self.item_embeddings is not None:
            u_profile = self._build_profile_online(user_interactions)
        else:
            u_profile = None

        return u_idx, user_ratings, u_profile

    def _build_profile_online(self, user_interactions):
        """实时构建未知用户的主题画像（冷启动场景）"""
        dim = self.item_embeddings.shape[1]
        profile = np.zeros(dim)
        total_w = 0.0
        for inter in user_interactions:
            p = self.poem_id_map.get(inter["poem_id"])
            if p is None:
                continue
            r = inter.get("rating", 3.0)
            w = r - self.global_mean
            w = 0.15 if abs(w) < 0.3 else w
            profile += w * self.item_embeddings[p]
            total_w += abs(w)
        if total_w > 1e-8:
            profile /= total_w
        norm = np.linalg.norm(profile)
        return profile / norm if norm > 1e-8 else profile

    def _semantic_scores(self, u_profile, exclude_set):
        """
        语义评分：cosine(用户主题画像, 物品语义向量)
        item_embeddings 已归一化 → 点积即余弦相似度
        """
        if u_profile is None or np.linalg.norm(u_profile) < 1e-8:
            return None
        scores = self.item_embeddings @ u_profile              # (items,)
        for pid in exclude_set:
            p = self.poem_id_map.get(pid)
            if p is not None:
                scores[p] = -np.inf
        return scores

    def _cf_scores(self, user_ratings, exclude_set):
        """Item-CF 评分（向量化）"""
        rated = np.where(user_ratings > 0)[0]
        if len(rated) == 0:
            return None
        sims = self.enhanced_sim[:, rated]                     # (items, rated)
        r_v  = user_ratings[rated]
        pos  = sims > 0
        w_sum   = np.where(pos, sims * r_v, 0.0).sum(axis=1)
        sim_sum = np.where(pos, np.abs(sims), 0.0).sum(axis=1) + 1e-8
        scores  = w_sum / sim_sum
        # mask rated & exclude
        scores[rated] = -np.inf
        for pid in exclude_set:
            p = self.poem_id_map.get(pid)
            if p is not None:
                scores[p] = -np.inf
        return scores

    @staticmethod
    def _minmax_valid(arr):
        """归一化到 [0,1]，-inf 处保持 -inf"""
        if arr is None:
            return None
        valid = arr[arr > -np.inf]
        if len(valid) == 0:
            return arr
        lo, hi = valid.min(), valid.max()
        return np.where(arr > -np.inf, (arr - lo) / (hi - lo + 1e-8), -np.inf)

    def recommend(self, user_interactions, exclude_ids=None, top_k=10):
        exclude_set = set(exclude_ids or []) | {i["poem_id"] for i in user_interactions}
        u_idx, user_ratings, u_profile = self._get_user_state(user_interactions)
        n_rated = int((user_ratings > 0).sum())

        # ── 自适应三路权重 ─────────────────────────────────────────────
        # 评分越少 → 语义主导；评分越多 → 协同信号越可靠
        # sem_w : 纯内容语义
        # ucf_w : 语义邻域 User-CF（找相似用户的真实评分）
        # icf_w : Item-CF 协同过滤
        if n_rated < 5:
            sem_w, ucf_w, icf_w = 0.80, 0.15, 0.05
        elif n_rated < 15:
            sem_w, ucf_w, icf_w = 0.55, 0.30, 0.15
        elif n_rated < 30:
            sem_w, ucf_w, icf_w = 0.40, 0.35, 0.25
        else:
            sem_w, ucf_w, icf_w = 0.30, 0.40, 0.30

        sem  = self._minmax_valid(self._semantic_scores(u_profile, exclude_set))
        ucf  = self._minmax_valid(self._user_cf_semantic_scores(u_profile, user_ratings, exclude_set))
        icf  = self._minmax_valid(self._cf_scores(user_ratings, exclude_set))

        # 三路融合：优先用有效信号，按权重加权
        def safe_add(combined, score, weight):
            if score is None:
                return combined, 0.0
            valid = score > -np.inf
            if combined is None:
                return np.where(valid, weight * score, -np.inf), weight
            both = (combined > -np.inf) & valid
            only_new = (combined <= -np.inf) & valid
            only_old = (combined > -np.inf) & ~valid
            return (np.where(both, combined + weight * score,
                    np.where(only_new, weight * score,
                    np.where(only_old, combined, -np.inf)))), weight

        combined, w_used = None, 0.0
        for score, w in [(sem, sem_w), (ucf, ucf_w), (icf, icf_w)]:
            combined, w_added = safe_add(combined, score, w)
            w_used += w_added if score is not None else 0

        if combined is None or (combined > -np.inf).sum() == 0:
            return self._popular_fallback(top_k, exclude_set)

        valid_idx = np.where(combined > -np.inf)[0]
        if len(valid_idx) == 0:
            return self._popular_fallback(top_k, exclude_set)

        top_idx = valid_idx[np.argsort(combined[valid_idx])[::-1][:top_k]]
        return [{"poem_id": self.poem_ids[i], "score": float(combined[i])} for i in top_idx]

    def predict_rating(self, user_interactions, poem_id):
        p_idx = self.poem_id_map.get(poem_id)
        if p_idx is None:
            return self.global_mean

        _, user_ratings, u_profile = self._get_user_state(user_interactions)
        rated = np.where(user_ratings > 0)[0]

        if len(rated) > 0:
            sims = self.enhanced_sim[p_idx, rated]
            r_v  = user_ratings[rated]
            pos  = sims > 0
            if pos.sum() > 0:
                pred = np.dot(sims[pos], r_v[pos]) / (np.abs(sims[pos]).sum() + 1e-8)
                return float(np.clip(pred, 1.0, 5.0))

        # 语义降级：用最相似物品的评分
        if u_profile is not None and len(rated) > 0:
            sem_sims = self.item_embeddings[p_idx] @ self.item_embeddings[rated].T
            best = rated[np.argmax(sem_sims)]
            return float(np.clip(user_ratings[best], 1.0, 5.0))

        return self.global_mean

    def _popular_fallback(self, top_k, exclude_ids=None):
        exclude_ids = exclude_ids or set()
        cnt = Counter()
        for inter in self.interactions:
            if inter["poem_id"] not in exclude_ids:
                cnt[inter["poem_id"]] += inter.get("rating", 3.0)
        return [{"poem_id": pid, "score": float(s)} for pid, s in cnt.most_common(top_k)]

# 添加基线模型
class RandomRecommender:
    def __init__(self, all_poem_ids):
        self.all_poem_ids = all_poem_ids

    def recommend(self, user_interactions, exclude_ids, top_k=10):
        candidates = list(set(self.all_poem_ids) - set(exclude_ids))
        np.random.shuffle(candidates)
        return [{"poem_id": pid, "score": np.random.random()} for pid in candidates[:top_k]]

    def predict_rating(self, user_interactions, poem_id):
        return np.random.uniform(1, 5)

class PopularRecommender:
    def __init__(self, popularity_scores):
        self.popularity_scores = popularity_scores  # dict of poem_id: score

    def recommend(self, user_interactions, exclude_ids, top_k=10):
        candidates = sorted(
            [(pid, score) for pid, score in self.popularity_scores.items() if pid not in exclude_ids],
            key=lambda x: x[1],
            reverse=True
        )
        return [{"poem_id": pid, "score": score} for pid, score in candidates[:top_k]]

    def predict_rating(self, user_interactions, poem_id):
        return self.popularity_scores.get(poem_id, 3.0)

## 8. 训练模型

In [12]:
print("\n5. 训练模型")
print("-" * 60)

# 计算全局流行度作为基线
popularity = Counter(i["poem_id"] for i in train_interactions if i["rating"] >= 4)
popularity = {pid: count / max(popularity.values()) * 5 for pid, count in popularity.items()}

# 初始化基线
random_rec = RandomRecommender([p["id"] for p in poems])
popular_rec = PopularRecommender(popularity)

# 核心模型训练（假设已定义类）
item_cf = ItemBasedCFRecommender()
item_cf.fit(train_filtered, [p["id"] for p in poems_filtered])   # ← 改这里！

content_rec = ContentBasedRecommender()
content_rec.fit(poems_filtered)   # Content-Based 的 fit 是 (items, interactions=None)，所以传 poems_filtered 就行

bertopic_cf = BERTopicEnhancedCF()
bertopic_cf.fit(poems_filtered, train_filtered)   # BERTopicEnhancedCF 是 (poems, interactions)，顺序正确


5. 训练模型
------------------------------------------------------------
[Item-CF] 相似度矩阵构建完成（向量化）
[Item-CF] 评分矩阵构建完成: (943, 1019)
[Item-CF] 矩阵密度: 8.04%
[ContentBased] 物品ID到索引映射构建完成: 1019 条
[ContentBased] TF-IDF矩阵: (1019, 1000)
[BERTopicEnhancedCF] 初始化组件...
[BERTopicEnhancedCF] 构建评分矩阵...
[BERTopicEnhancedCF] 评分矩阵: (943, 1019)
[BERTopicEnhancedCF] 计算Item-CF相似度...
[BERTopicEnhancedCF] Item相似度矩阵完成
[BERTopicEnhancedCF] 计算User-CF相似度...
[BERTopicEnhancedCF] User相似度矩阵完成
[BERTopicEnhancedCF] 训练BERTopic模型...
[BERTopic] 设备: cpu
[BERTopic] 加载embedding模型...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[BERTopic] 为 1019 个物品生成向量 (GPU=False)...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

[BERTopic] 主题建模跳过: empty vocabulary; perhaps the documents only contain stop words
[BERTopic] 向量已缓存
[BERTopicEnhancedCF] BERTopic向量矩阵: (1019, 384)
[BERTopicEnhancedCF] 计算混合用户相似度 (Top-K + 置信度过滤)...
[BERTopicEnhancedCF] 混合用户相似度完成 (α=0.6)
[BERTopicEnhancedCF] 计算增强相似度矩阵...
[BERTopicEnhancedCF] 融合完成: 0.5×Item-CF + 0.2×主题
[BERTopicEnhancedCF] 训练完成


## 9. 评估指标（优化版）

In [13]:
def evaluate_recommender(recommender, train_interactions, test_interactions, poems, top_k=10, rating_threshold=4, is_cold=False):
    """评估推荐系统性能（优化版）"""
    user_train = defaultdict(list)
    for inter in train_interactions:
        user_train[inter["user_id"]].append(inter)

    user_test = defaultdict(set)
    user_test_ratings = defaultdict(dict)
    for inter in test_interactions:
        user_test_ratings[inter["user_id"]][inter["poem_id"]] = inter["rating"]
        if inter["rating"] >= rating_threshold:
            user_test[inter["user_id"]].add(inter["poem_id"])

    precisions, recalls, f1_scores, ndcgs = [], [], [], []
    predicted_ratings, actual_ratings = [], []
    all_recommended = set()

    test_users = list(user_test.keys())
    if is_cold:
        test_users = [u for u in test_users if len(user_train[u]) < 5]

    for user_id in test_users:
        if user_id not in user_train:
            continue

        seen = set(i["poem_id"] for i in user_train[user_id])
        try:
            recs = recommender.recommend(user_train[user_id], exclude_ids=seen, top_k=top_k)
        except Exception:
            continue

        rec_items = [r["poem_id"] for r in recs]
        rec_scores = [r["score"] for r in recs]
        all_recommended.update(rec_items)

        relevant = list(user_test[user_id])
        if not relevant:
            continue

        hits = [1 if pid in user_test[user_id] else 0 for pid in rec_items]
        precision = sum(hits) / top_k
        recall = sum(hits) / len(relevant)
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

        # NDCG
        ideal_hits = sorted(hits, reverse=True)
        ndcg = ndcg_score([ideal_hits], [hits])

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        ndcgs.append(ndcg)

        # ★ MAE：如果模型实现了 predict_all_ratings，则批量预测（快 100x）
        if hasattr(recommender, 'predict_all_ratings') and hasattr(recommender, 'item_id_to_idx'):
            # ContentBased 路径
            _idx_map = recommender.item_id_to_idx
            _all_preds = recommender.predict_all_ratings(user_train[user_id])
            for poem_id, actual in user_test_ratings[user_id].items():
                p_idx = _idx_map.get(poem_id)
                if p_idx is not None:
                    predicted_ratings.append(float(_all_preds[p_idx]))
                    actual_ratings.append(actual)
        elif hasattr(recommender, 'predict_all_ratings') and hasattr(recommender, 'poem_id_map'):
            # SemanticCF 路径
            _idx_map = recommender.poem_id_map
            _all_preds = recommender.predict_all_ratings(user_train[user_id])
            for poem_id, actual in user_test_ratings[user_id].items():
                p_idx = _idx_map.get(poem_id)
                if p_idx is not None:
                    predicted_ratings.append(float(_all_preds[p_idx]))
                    actual_ratings.append(actual)
        else:
            for poem_id, actual in user_test_ratings[user_id].items():
                pred = recommender.predict_rating(user_train[user_id], poem_id)
                predicted_ratings.append(pred)
                actual_ratings.append(actual)

    mae = np.mean(np.abs(np.array(predicted_ratings) - np.array(actual_ratings))) if predicted_ratings else np.nan
    coverage = len(all_recommended) / len(poems) if poems else 0

    return {
        "precision": np.mean(precisions) if precisions else 0,
        "recall": np.mean(recalls) if recalls else 0,
        "f1": np.mean(f1_scores) if f1_scores else 0,
        "ndcg": np.mean(ndcgs) if ndcgs else 0,
        "mae": mae,
        "coverage": coverage,
        "n_users": len(precisions),
    }

## 10. 运行实验

In [ ]:
K_VALUES = [5, 10, 15, 20]

results = {
    "Random": {},
    "Popular": {},
    "Item-CF": {},
    "Content-Based": {},
    "BERTopic-Enhanced CF": {}
}

# ★ Fix: 深拷贝，避免 results 和 results_cold 共享内层 dict 对象
results_cold = {name: {} for name in results}

models = {
    "Random": random_rec,
    "Popular": popular_rec,
    "Item-CF": item_cf,  # 假设已初始化
    "Content-Based": content_rec,
    "BERTopic-Enhanced CF": bertopic_cf,
}

print("\n6. 评估模型")
print("-" * 60)

for k in K_VALUES:
    print(f"\nK = {k}")
    for name, model in models.items():
        print(f"  评估 {name}...", end=" ")
        start_time = time.time()
        metrics = evaluate_recommender(model, train_interactions, test_interactions, poems, top_k=k)
        metrics_cold = evaluate_recommender(model, train_interactions, test_interactions, poems, top_k=k, is_cold=True)
        elapsed = time.time() - start_time
        results[name][k] = metrics
        results_cold[name][k] = metrics_cold
        print(f"P={metrics['precision']:.4f}, R={metrics['recall']:.4f}, F1={metrics['f1']:.4f}, NDCG={metrics['ndcg']:.4f}, MAE={metrics['mae']:.4f}, Cov={metrics['coverage']:.4f} ({elapsed:.1f}s)")


6. 评估模型
------------------------------------------------------------

K = 5
  评估 Random... P=0.0079, R=0.0037, F1=0.0045, NDCG=0.0239, MAE=1.3789, Cov=0.9322 (1.0s)
  评估 Popular... P=0.0922, R=0.0428, F1=0.0519, NDCG=0.2175, MAE=2.6221, Cov=0.0279 (1.2s)
  评估 Item-CF... P=0.0152, R=0.0047, F1=0.0064, NDCG=0.0362, MAE=0.8596, Cov=0.3942 (19.2s)
  评估 Content-Based... P=0.0307, R=0.0150, F1=0.0173, NDCG=0.0830, MAE=0.9709, Cov=0.1296 (62.6s)
  评估 BERTopic-Enhanced CF... 

## 11. 结果汇总

In [ ]:
print("\n" + "=" * 60)
print("7. 实验结果汇总")
print("=" * 60)

K = 10
# ★ Fix: 找到实际跑过的最大 K，防止 K=10 未在 K_VALUES 中导致 KeyError
available_k = [k for k in K_VALUES if all(k in results[n] for n in results if results[n])]
K_display = K if K in available_k else (available_k[-1] if available_k else None)

if K_display is None:
    print("  ⚠ 尚无评估结果可展示，请先运行 Cell 20（评估模型）")
else:
    print(f"\n【整体数据集 K = {K_display} 性能对比】")
    print("-" * 80)
    print(f"{{'算法':<20}} {{'Precision':>10}} {{'Recall':>10}} {{'F1':>10}} {{'NDCG':>10}} {{'MAE':>10}} {{'Coverage':>10}}")
    print("-" * 80)
    for name in results.keys():
        if K_display not in results[name]:
            continue
        m = results[name][K_display]
        mae_str = f"{{m['mae']:.4f}}" if not np.isnan(m["mae"]) else "N/A"
        print(f"{{name:<20}} {{m['precision']:>10.4f}} {{m['recall']:>10.4f}} {{m['f1']:>10.4f}} {{m['ndcg']:>10.4f}} {{mae_str:>10}} {{m['coverage']:>10.4f}}")
    print("-" * 80)

    print(f"\n【冷启动子集 K = {K_display} 性能对比】")
    print("-" * 80)
    print(f"{{'算法':<20}} {{'Precision':>10}} {{'Recall':>10}} {{'F1':>10}} {{'NDCG':>10}} {{'MAE':>10}} {{'Coverage':>10}}")
    print("-" * 80)
    for name in results_cold.keys():
        if K_display not in results_cold[name]:
            continue
        m = results_cold[name][K_display]
        mae_str = f"{{m['mae']:.4f}}" if not np.isnan(m["mae"]) else "N/A"
        print(f"{{name:<20}} {{m['precision']:>10.4f}} {{m['recall']:>10.4f}} {{m['f1']:>10.4f}} {{m['ndcg']:>10.4f}} {{mae_str:>10}} {{m['coverage']:>10.4f}}")
    print("-" * 80)

## 12. 可视化

In [ ]:
print("\n8. 生成可视化图表")
print("-" * 60)

# 图1: 指标随K变化曲线
fig, axes = plt.subplots(3, 2, figsize=(16, 12))
metrics_list = ["precision", "recall", "f1", "ndcg", "mae", "coverage"]
titles = ["Precision@K", "Recall@K", "F1-Score@K", "NDCG@K", "MAE@K", "Coverage@K"]
colors = ["#2ecc71", "#3498db", "#e74c3c", "#9b59b6", "#f1c40f"]

for ax, metric, title in zip(axes.flat, metrics_list, titles):
    for name, color in zip(results.keys(), colors):
        vals = [results[name][k][metric] for k in K_VALUES]
        ax.plot(K_VALUES, vals, marker="o", linewidth=2, markersize=8, label=name, color=color)

    ax.set_xlabel("K", fontsize=12)
    ax.set_ylabel(metric.capitalize(), fontsize=12)
    ax.set_title(title, fontsize=14, fontweight="bold")
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xticks(K_VALUES)

plt.tight_layout()
plt.savefig("metrics_comparison.png", dpi=150, bbox_inches="tight")
plt.show()
print("保存: metrics_comparison.png")

# 图2: K=10柱状图
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(results))
width = 0.15

for i, (metric, color) in enumerate(zip(metrics_list[:-1], ["#2ecc71", "#3498db", "#e74c3c", "#9b59b6", "#f1c40f"])):
    vals = [results[name][K][metric] for name in results.keys()]
    bars = ax.bar(x + i * width, vals, width, label=metric.capitalize(), color=color)

    for bar in bars:
        height = bar.get_height()
        ax.annotate(f"{height:.3f}", xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha="center", va="bottom", fontsize=9)

ax.set_xlabel("Algorithm", fontsize=12)
ax.set_ylabel("Score", fontsize=12)
ax.set_title(f"Performance Comparison @ K={K}", fontsize=14, fontweight="bold")
ax.set_xticks(x + width * 2)
ax.set_xticklabels(list(results.keys()), fontsize=10)
ax.legend()
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.savefig("k10_comparison.png", dpi=150, bbox_inches="tight")
plt.show()
print("保存: k10_comparison.png")

## 13. 实验结论

In [ ]:
print("\n" + "=" * 60)
print("9. 实验结论")
print("=" * 60)

best_f1 = max(results.keys(), key=lambda x: results[x][K]["f1"])
print(f"\n最佳F1: {best_f1} (F1={results[best_f1][K]['f1']:.4f})")

if "BERTopic-Enhanced CF" in results:
    bert_f1 = results["BERTopic-Enhanced CF"][K]["f1"]
    itemcf_f1 = results["Item-CF"][K]["f1"]
    content_f1 = results["Content-Based"][K]["f1"]

    if itemcf_f1 > 0:
        imp1 = (bert_f1 - itemcf_f1) / itemcf_f1 * 100
        print(f"BERTopic-Enhanced CF 相比 Item-CF F1提升: {imp1:+.2f}%")

    if content_f1 > 0:
        imp2 = (bert_f1 - content_f1) / content_f1 * 100
        print(f"BERTopic-Enhanced CF 相比 Content-Based F1提升: {imp2:+.2f}%")

print("\n" + "=" * 60)
print("实验完成!")
print("=" * 60)